In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ast

In [3]:


# Read the CSV file, ensuring no extra index is added
df_songs_chart_usa = pd.read_csv('/Users/fynnersatz/Desktop/python/hitster/django/music_game/game/data/raw_de/chart_data_de.csv', index_col=None)
df_songs_spotify_usa = pd.read_csv('/Users/fynnersatz/Desktop/python/hitster/django/music_game/game/data/raw_de/spotify_data_de.csv', index_col=None)
df_songs_spotify_usa['artist_spotify'] = df_songs_spotify_usa['artist_spotify'].apply(ast.literal_eval)
df_songs_spotify_usa['country'] = "usa"


# Read the CSV file, ensuring no extra index is added
df_songs_chart_de = pd.read_csv('/Users/fynnersatz/Desktop/python/hitster/django/music_game/game/data/raw_usa/chart_data_usa.csv', index_col=None)
df_songs_spotify_de = pd.read_csv('/Users/fynnersatz/Desktop/python/hitster/django/music_game/game/data/raw_usa/spotify_data_usa.csv', index_col=None)
df_songs_spotify_de['artist_spotify'] = df_songs_spotify_de['artist_spotify'].apply(ast.literal_eval)
df_songs_spotify_de['country'] = "de"
# Concatenate spaltenweise
df_songs_de = pd.concat([df_songs_chart_de, df_songs_spotify_de], axis=1)
df_songs_de = df_songs_de[df_songs_de["match_score"] >150]

df_songs_usa = pd.concat([df_songs_chart_usa, df_songs_spotify_usa], axis=1)
df_songs_usa = df_songs_usa[df_songs_usa["match_score"] >150]



In [4]:
# Funktion zur Sicherstellung der Gleichheit für Listen oder einfache Werte
def ensure_unique(series, col_name):
    if series.apply(lambda x: isinstance(x, list)).all():  # Prüfen, ob alle Werte Listen sind
        unique_values = series.apply(tuple).unique()  # Listen in Tupel umwandeln für .unique()
    else:
        unique_values = series.unique()
    if len(unique_values) > 1:
        raise ValueError(f"Spalte '{col_name}' hat mehrere unterschiedliche Werte in einer Gruppe: {unique_values}")
    return unique_values[0]



In [5]:
# Aggregationslogik definieren
aggregations_country = {
    "artists": lambda x: x.iloc[0],  # Take the first element of the group
    "artist_spotify": lambda x: ensure_unique(x, "artist_spotify"),
    "title": lambda x: x.iloc[0],   # Take the first element of the group
    "title_spotify": lambda x: ensure_unique(x, "title_spotify"),
    "match_score": lambda x: x.iloc[0],  # Take the first element of the group
    "year_released": lambda x: x.iloc[0],  # Take the first element of the group
    "year_charts": lambda x: x.min() if len(x) == len(set(x)) else ValueError(f"'year_charts' hat doppelte Werte: {x.tolist()}"),
    "points": "sum",
    "pos": "min",
    "max_pos": "min",
    "weeks_top10": "sum",
    "image": lambda x: ensure_unique(x, "image"),
    "country": lambda x: ensure_unique(x, "country"),
}

# Grouping und Aggregation
try:
    # `id_spotify` wird als Index verwendet und anschließend zurückgesetzt
    df_songs_de_sorted = df_songs_de.sort_values(by=["id_spotify", "match_score"], ascending=[True, False])
    df_songs_de = df_songs_de_sorted.groupby("id_spotify").agg(aggregations_country).reset_index()
except ValueError as e:
    print("Fehler bei der Gruppierung:", e)

try:
    # `id_spotify` wird als Index verwendet und anschließend zurückgesetzt
    df_songs_usa_sorted = df_songs_usa.sort_values(by=["id_spotify", "match_score"], ascending=[True, False])
    df_songs_usa = df_songs_usa_sorted.groupby("id_spotify").agg(aggregations_country).reset_index()
except ValueError as e:
    print("Fehler bei der Gruppierung:", e)

In [6]:
df_songs = pd.concat([df_songs_de, df_songs_usa], axis=0)

In [7]:
# Aggregationslogik definieren
aggregations_total = {
    "artists": lambda x: x.iloc[0],  # Take the first element of the group
    "artist_spotify": lambda x: ensure_unique(x, "artist_spotify"),
    "title": lambda x: x.iloc[0],   # Take the first element of the group
    "title_spotify": lambda x: ensure_unique(x, "title_spotify"),
    "match_score": lambda x: x.iloc[0],  # Take the first element of the group
    "year_released": lambda x: x.iloc[0],  # Take the first element of the group
    "year_charts": lambda x: x.iloc[0],  # Take the first element of the group
    "points": "sum",
    "pos": "min",
    "max_pos": "min",
    "weeks_top10": "sum",
    "image": lambda x: ensure_unique(x, "image"),
    "country": lambda x: x.iloc[0],  # Take the first element of the group
}
try:
    # `id_spotify` wird als Index verwendet und anschließend zurückgesetzt
    df_songs_sorted = df_songs.sort_values(by=["id_spotify", "year_released", "match_score"], ascending=[True, True, False])
    df_songs_filtered = df_songs_sorted.groupby("id_spotify").agg(aggregations_total).sort_values(["year_released", "points"], ascending=[False, False]).reset_index()
except ValueError as e:
    print("Fehler bei der Gruppierung:", e)

In [9]:
top_10_p_year = df_songs_filtered.groupby("year_released", group_keys=False).apply(lambda group: group.nlargest(10, "points"))
top_10_p_year.to_csv("song_data.csv", index=False)

/var/folders/xx/yw34cppn2cv3kmh21dv6xcnw0000gp/T/ipykernel_20681/2590640457.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_10_p_year = df_songs_filtered.groupby("year_released", group_keys=False).apply(lambda group: group.nlargest(10, "points"))


In [ ]:
pd.set_option("display.max_rows", 500)
top_10_p_year.sort_values(["year_released", "points"], ascending=[False, False]).head(100)

## Plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def create_boxplot(df, x_col, y_col, title=None, figsize=(10, 6)):
    """
    Create a box plot from a pandas DataFrame.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame
    x_col (str): Name of the integer column for x-axis
    y_col (str): Name of the numeric column for y-axis
    title (str, optional): Plot title
    figsize (tuple, optional): Figure size as (width, height)
    """
    # Create figure and axis
    plt.figure(figsize=figsize)
    
    # Create box plot using seaborn
    sns.boxplot(data=df, x=x_col, y=y_col, fli)
    
    # Customize plot
    plt.title(title or f'Box Plot of {y_col} by {x_col}')
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    
    # Rotate x-axis labels if there are many categories
    plt.xticks(rotation=45 if len(df[x_col].unique()) > 10 else 0)
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    # Show plot
    plt.show()



In [ ]:
create_boxplot(df_songs_filtered, "year_released", "points")

In [ ]:

df_songs_filtered.groupby(by="year_released").agg({"points": "median"}).plot()

In [ ]:
df_songs_filtered.sort_values("points", ascending=False).head(200).groupby(by="year_released").agg({"points": "median"}).plot()

In [ ]:
df_songs_filtered.hist(column="year_released", bins=len(df_songs_filtered["year_released"].unique()))

In [14]:
from geopy.geocoders import Nominatim
import time

# Initialisierung des Nominatim-Geocoders
geolocator = Nominatim(user_agent="geoapi_exercises")

# Adresse, für die die Koordinaten ermittelt werden sollen
adresse = "Amselweg 18, Isenbüttel, Deutschland"

# Geocodierung der Adresse und zeit für die anfrage
start = time.time()
location = geolocator.geocode(adresse)
end = time.time()
print(f"Zeit für die Anfrage: {end - start:.2f} Sekunden")

# Ausgabe der Ergebnisse
if location:
    print(f"Adresse: {location.address}")
    print(f"Breitengrad: {location.latitude}")
    print(f"Längengrad: {location.longitude}")
else:
    print("Adresse konnte nicht gefunden werden!")


Zeit für die Anfrage: 0.23 Sekunden
Adresse: 18, Amselweg, Isenbüttel, Samtgemeinde Isenbüttel, Gifhorn, Niedersachsen, 38550, Deutschland
Breitengrad: 52.4300349
Längengrad: 10.581476200541125
